# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **New York, New York, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **New York, New York, United States** to Ann Arbor, USA. In that case at least one source file must be about **New York, New York, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **New York, New York, United States** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

columns_clean = ['year','wins', 'losses','success_rate']

giants = pd.read_csv('giants.csv')
giants['year'] = giants['Season'].apply(str)
giants['wins'] = giants['Wins']
giants['losses'] = giants['Losses']
giants['success_rate'] = giants['Wins'] / (giants['Wins'] + giants['Losses'])
giants = giants[columns_clean]
giants.set_index('year', inplace=True)

knicks = pd.read_csv('knicks.csv', encoding = "ISO-8859-1")
knicks.dropna(subset=['Season'], inplace=True)
knicks['year'] = knicks['Season'].apply(lambda x: x[:4])
knicks['wins'] = knicks['W[c]']
knicks['losses'] = knicks['L[c]']
knicks['success_rate'] = knicks['W[c]'] / (knicks['W[c]'] + knicks['L[c]'])
knicks = knicks[columns_clean]
knicks.set_index('year', inplace=True)

rangers = pd.read_csv('rangers.csv', encoding = "ISO-8859-1", skiprows=[0])
rangers.dropna(subset=['Unnamed: 0'], inplace=True)
rangers['year'] = rangers['Unnamed: 0'].apply(lambda x: x[:4])
rangers['wins'] = rangers['W']
rangers['losses'] = rangers['L']
rangers['success_rate'] = rangers['W'] / (rangers['W'] + rangers['L'])
rangers = rangers[columns_clean]
rangers.set_index('year', inplace=True)

yankees = pd.read_csv('yankees.csv', encoding = "ISO-8859-1", skiprows=[0])
yankees.dropna(subset=['MLB season'], inplace=True)
yankees['year'] = yankees['MLB season'].apply(lambda x: x[:4])
yankees['wins'] = yankees['Wins[11]']
yankees['losses'] = yankees['Losses[11]']
yankees['success_rate'] = yankees['Wins[11]'] / (yankees['Wins[11]'] + yankees['Losses[11]'])
yankees = yankees[columns_clean]
yankees.set_index('year', inplace=True)

gathered_data = giants
gathered_data = gathered_data.join(knicks, rsuffix='_knicks')
gathered_data = gathered_data.join(rangers, rsuffix='_rangers')
gathered_data = gathered_data.join(yankees, rsuffix='_yankees')
gathered_data.rename(columns={'success_rate': 'success_rate_giants', 'wins': 'wins_giants', 'losses': 'losses_giants'}, inplace=True)
gathered_data.fillna(method='ffill', inplace=True)
gathered_data.dropna(inplace=True)
gathered_data = gathered_data.rolling(window=15).mean()

all_stats = [np.sum(gathered_data['wins_giants']) / (np.sum(gathered_data['wins_giants']) + np.sum(gathered_data['losses_giants'])),
            np.sum(gathered_data['wins_knicks']) / (np.sum(gathered_data['wins_knicks']) + np.sum(gathered_data['losses_knicks'])),
            np.sum(gathered_data['wins_rangers']) / (np.sum(gathered_data['wins_rangers']) + np.sum(gathered_data['losses_rangers'])),
            np.sum(gathered_data['wins_yankees']) / (np.sum(gathered_data['wins_yankees']) + np.sum(gathered_data['losses_yankees']))]
all_stats = pd.Series(all_stats, index=['Football', 'Basketball', 'Hockey', 'Baseball'])
all_stats.sort_values(ascending=True, inplace=True)

plt.style.use('seaborn-colorblind')

fig, subaxes = plt.subplots(1, 2, figsize=(15,5))

foot = subaxes[0].plot(gathered_data['success_rate_giants'], label='Football', alpha=0.8, linewidth=5)
basket = subaxes[0].plot(gathered_data['success_rate_knicks'], label='Basketball', alpha=0.8, linewidth=5)
hockey = subaxes[0].plot(gathered_data['success_rate_rangers'], label='Hockey', alpha=0.8, linewidth=5)
baseball = subaxes[0].plot(gathered_data['success_rate_yankees'], label='Baseball', alpha=0.8, linewidth=5)
subaxes[0].legend()
subaxes[0].set_title('New York sports win rate in % \n (10 years moving average)')
subaxes[0].set_ylabel('Win rate in % (10 years moving average)')
subaxes[0].set_xlabel('Seasons')
subaxes[0].set_ylim(0.25,0.75)
subaxes[0].spines['top'].set_visible(False)
subaxes[0].spines['right'].set_visible(False)

subaxes[1].bar(range(len(all_stats)), all_stats, alpha=0.8, color=[basket[0].get_color(), foot[0].get_color(), hockey[0].get_color(), baseball[0].get_color()])
subaxes[1].set_xticks(range(len(all_stats)))
subaxes[1].set_xticklabels(all_stats.index)
subaxes[1].set_ylim(0.25,0.75)
subaxes[1].spines['top'].set_visible(False)
subaxes[1].spines['right'].set_visible(False)
subaxes[1].spines['left'].set_visible(False)
subaxes[1].get_yaxis().set_visible(False)
subaxes[1].set_title('New York sports average win rate all time')

labels = ['{perc:.1%}'.format(perc=d) for d in all_stats.values]
for rect, label, plot in zip(subaxes[1].patches, labels, [basket, foot, hockey, baseball]):
    subaxes[1].text(rect.get_x() + rect.get_width() / 2, rect.get_height() +0.02, label, ha='center', color=plot[0].get_color())

plt.show()

AttributeError: 'Rectangle' object has no attribute 'get_color'